In [9]:
import os
import numpy as np
import pandas as pd

from glob import glob
from src.utils import read_image, read_hparams
from tqdm import tqdm
from models.clips import *
from cfg import CFG
from src.data_module import DataModuleForCLIP

- Copy and paste your weight path and `hparams.yaml`

In [2]:

weight = "lightning_logs/version_13/checkpoints/wallpaper_fault-epoch=199-val_F1Score=0.87.ckpt"
hparams_path = "/home/hm086/joono/DACON/CLIP_for_classificaion_with_descriptions/lightning_logs/version_13/hparams.yaml"

hparams = read_hparams(hparams_path)
label2cls = {label: cls for cls, label in hparams['class_label'].items()}

- Enter num_classes (You can check this in `lightning_logs/*/hparams.yaml`)

In [3]:
CFG.num_classes = 19

- Select corresponding model

In [4]:
#########################################
# Select model
#########################################
# model = CLIP_ViT_base_patch32()
# model = CLIP_ViT_large_patch14()
model = CLIP_ConvNeXt_large_patch14.load_from_checkpoint(weight)
# model = CLIP_ConvNeXt_base_patch32()
# model = CLIP_ResNet50_large_patch14()
#########################################

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


- inference example code

In [15]:
test_files = glob('/path/to/wallpaper_fault_classification/test/*')

model.eval()

with torch.no_grad():
    ids, classes = [], []
    for test in tqdm(test_files):
        
        image = read_image(test)
        image = CFG.val_transforms(image).to("cuda")
        
        _, y_hat, _ = model(image.unsqueeze(0))
        cls_hat = torch.argmax(y_hat)
        
        cls = label2cls[cls_hat.item()]
        id = f"TEST_{os.path.basename(test).split('.')[0]}"
        
        ids.append(id)
        classes.append(cls)
    
df = pd.DataFrame({
    "id": ids,
    "label": classes
})

df = df.sort_values("id")
df.to_csv("submission17.csv", index=False, encoding="utf-8-sig")